# Formation energy prediction task 01

Подготовим прогноз энергии образования с помощью модели GNN. 

Let's prepare a prediction of the formation energy using the GNN model.

## Import libraries Импорт библиотек

In [1]:
# common libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tqdm, os
import seaborn as sns

from sklearn.metrics import r2_score, mean_absolute_error
from tqdm import tqdm

# torch libraries
from torch.utils.data import DataLoader

In [22]:
from torch_geometric.loader import DataLoader as DataLoader_geometric

In [17]:
import torch

In [7]:
# Torch geometric libraries
from torch_geometric.data import Data, Dataset

In [2]:
# import modules
from models.GNN_first import GCN

In [3]:
# assistive libraries
plt.rcParams.update(plt.rcParamsDefault) # Сброс настроек
import openpyxl #, работа с excel

## Loading dataset Загружаем данные

In [ ]:
# загружаем датафрейм df_Fm-3m.xlsx
PATH_FOR_LOAD = r'.\data\datafraims\df_Fm-3m.xlsx'
df_load = pd.read_excel(PATH_FOR_LOAD)

In [5]:
# смотрим
df_load.tail(2)

,composition,structure,formation_energy_per_atom
1255,Zr1 Sn3,Full Formula (Zr1 Sn3)\nReduced Formula: ZrSn3...,0.115889
1256,Zr1 Zn1,Full Formula (Zr1 Zn1)\nReduced Formula: ZrZn\...,0.104294


In [ ]:
# создадим класс для загрузки графов из соответствующей папки репозитория  
class ProcessedDataset_new_dir(Dataset): # (наследник torch_geometric.data.Dataset)
    '''пользовательский класс ProcessedDataset для работы с предобработанными графами кристаллических структур, 
    сохраненными в файлах .pt, и выводит статистику о данных.'''

    # Инициализирует датасет:
    # root: Путь к папке с данными, transform/pre_transform/pre_filter: Опциональные функции для преобразования данных (не используются здесь).
    def __init__(self, root, new_dir_for_load_grahs, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.processed_new_dir = new_dir_for_load_grahs

    @property
    def processed_dir(self):
        return os.path.join(self.root, self.processed_new_dir)  # Кастомный путь
    
    # Возвращает список файлов .pt в папке processed
    @property
    def processed_file_names(self):
        file_names = []
        for i in os.listdir(self.processed_dir):
            if '.pt' in i:
                file_names.append(i)
        return file_names
    

    # Возвращает количество графов в датасете:
    def len(self):
        return len(self.processed_file_names)

    # Загружает граф по индексу idx
    def get(self, idx):
        data = torch.load(os.path.join(self.processed_dir, f'data_{idx}.pt'))
        return data

In [9]:
# загружаем датасет
dataset = ProcessedDataset_new_dir('./', './data/graphs_structures_Fm_3m')

In [ ]:
# смотрим тип и как читается ссылка
type(dataset), dataset.processed_new_dir

(__main__.ProcessedDataset_new_dir, './data/graphs_structures_Fm_3m')

In [19]:
# смотрим данные о датасете
print()
print(f'Dataset: {dataset}:')
print('=' * 79)
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')


Dataset: ProcessedDataset_new_dir(1257):
Number of graphs: 1257
Number of features: 5


## Splitting the data into train and test  Разбивка данных на обучающие и тестовые

Для первого варианта возьмём размер обучающей выборки 85%

In [ ]:
# 85% of data will be presented in the training set
train_fraction = 0.85

train_set_size = round(df_load.shape[0] * train_fraction)
print('Number of samples in the training set:', train_set_size)


# Get indexes for train and test # Получаем индексы обучающей и тестовой выборки

# Define train indices to compare differen models
train_idxs = df_load.sample(train_set_size).index

# Inverse selection of samples that are not in the train indices
test_idxs = df_load.loc[df_load.index.difference(train_idxs)].index

Number of samples in the training set: 1068


In [21]:
# получаем 
train_dataset = dataset[list(train_idxs)]
test_dataset = dataset[list(df_load.index.difference(train_idxs))]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 1068
Number of test graphs: 189


## Breakdown of data by batches Разбивка данных по пакетам

In [ ]:
# разбивка по пакетам
train_loader = DataLoader_geometric(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader_geometric(test_dataset, batch_size=228, shuffle=False)

# смотрим как разбились данные в train_loader
# we are looking at how the data is distributed in train_loader
for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 256
DataBatch(x=[9912, 5], y=[256, 1], pos=[9912, 3], lattice=[256, 9], edge_index=[2, 59472], batch=[9912], ptr=[257])

Step 2:
Number of graphs in the current batch: 256
DataBatch(x=[9504, 5], y=[256, 1], pos=[9504, 3], lattice=[256, 9], edge_index=[2, 57024], batch=[9504], ptr=[257])

Step 3:
Number of graphs in the current batch: 256
DataBatch(x=[9312, 5], y=[256, 1], pos=[9312, 3], lattice=[256, 9], edge_index=[2, 55872], batch=[9312], ptr=[257])

Step 4:
Number of graphs in the current batch: 256
DataBatch(x=[9520, 5], y=[256, 1], pos=[9520, 3], lattice=[256, 9], edge_index=[2, 57120], batch=[9520], ptr=[257])

Step 5:
Number of graphs in the current batch: 44
DataBatch(x=[1616, 5], y=[44, 1], pos=[1616, 3], lattice=[44, 9], edge_index=[2, 9696], batch=[1616], ptr=[45])



In [ ]:
# смотрим как разбились данные в test_loader
# we are looking at how the data is distributed in test_loader
for step, data in enumerate(test_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 189
DataBatch(x=[7912, 5], y=[189, 1], pos=[7912, 3], lattice=[189, 9], edge_index=[2, 47472], batch=[7912], ptr=[190])



## Initiating the model / Инициируем модель

In [28]:
# defining hyperparameters / определяем гиперпараметры 
hyperparameters = {'hidden_embeding':128,}

# инициируем модель 
model = GCN(hyperparameters=hyperparameters,)

TypeError: GCN.__init__() missing 1 required positional argument: 'dataset'

## Оптимизатор и функция потерь

In [ ]:
# создание оптимизатора Adam для обучения нейронной сети. Оптимизатор отвечает за обновление весов модели в процессе обучения.
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# добавил самостоятельно, насколько это тут необходимо пока не совсем понимаю
criterion = torch.nn.MSELoss()

## Функция запуска подели

In [27]:
# # Импорт метрик для оценки качества модели
# from sklearn.metrics import r2_score, mean_absolute_error
# import numpy as np

def run_model(model, epochs, train_loader, test_loader):
    """
    Основная функция для обучения и оценки модели.
    
    Параметры:
        model (torch.nn.Module): Нейронная сеть для обучения
        epochs (int): Количество эпох обучения
        train_loader (torch_geometric.loader.DataLoader): Загрузчик тренировочных данных
        test_loader (torch_geometric.loader.DataLoader): Загрузчик тестовых данных
    """
    
    def train():
        """
        Внутренняя функция для одной эпохи обучения.
        Возвращает суммарный лосс за эпоху.
        """
        # Переводим модель в режим обучения (важно для слоёв типа Dropout, BatchNorm)
        model.train()
        val_loss = 0  # Инициализируем переменную для накопления лосса

        # Итерируемся по батчам тренировочных данных
        for data in train_loader:
            # 1. Прямой проход (forward pass) - вычисление предсказаний модели
            out = model(data)

            # 2. Вычисление функции потерь между предсказаниями и истинными значениями
            loss = criterion(out, data['y'])

            # 3. Обратное распространение ошибки (backward pass) - вычисление градиентов
            loss.backward()

            # 4. Обновление параметров модели на основе вычисленных градиентов
            optimizer.step()

            # 5. Обнуление градиентов перед следующим батчем
            optimizer.zero_grad()

            # 6. Накопление лосса (detach() чтобы избежать накопления в вычислительном графе)
            val_loss += loss.detach().item()

        return val_loss  # Возвращаем суммарный лосс за эпоху


    def test(loader):
        """
        Внутренняя функция для оценки модели на переданном загрузчике данных.
        Возвращает среднее значение R2-score по всем батчам.
        """
        # Переводим модель в режим оценки (отключаем Dropout и т.д.)
        model.eval()
        r2 = []  # Список для хранения R2-score по каждому батчу

        # Итерируемся по батчам данных (без вычисления градиентов)
        for data in loader:
            # Получаем предсказания модели
            out = model(data)

            # Преобразуем предсказания и истинные значения в 1D массивы
            pred = out.detach().ravel()  # detach() чтобы не вычислять градиенты
            true = data['y'].ravel()

            # Вычисляем R2-score для текущего батча и сохраняем
            r2.append(r2_score(pred.numpy(), true.numpy()))

        # Возвращаем среднее значение R2-score по всем батчам
        return np.array(r2).mean()

    # Основной цикл обучения
    for epoch in range(1, epochs+1):
        # Одна эпоха обучения и получение лосса
        val_loss = train()
        
        # Вывод информации о текущей эпохе
        print(f'Epoch: {epoch:03d}, Val Loss: {val_loss:.4f}')

        # Каждые 10 эпох оцениваем модель на тренировочных и тестовых данных
        if epoch % 10 == 0:
            train_acc = test(train_loader)  # R2 на тренировочных данных
            test_acc = test(test_loader)   # R2 на тестовых данных
            print(f'Train R2: {train_acc:.4f}, Test R2: {test_acc:.4f}')